In [1]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange
from datetime import datetime, timedelta


In [2]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [3]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
pd.options.display.float_format = '{:.5f}'.format

In [4]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

# 데이터 수집

In [9]:
# 매장 입출판재 

start = time.time()

total_shop_stock_ori = '''
SELECT vfs.weekkey,vfs.shopkey ,vfs.partkey,vfs.scskey,vfs.brand,vfs.season,vfs.partcode,vfs.color,vfs.siz,vfs.week_takeout_qty,vfs.takeout_qty ,vfs.week_sale_qty ,vfs.sale_qty,vfs.week_sale_amt,vfs.stock_qty ,vfs.sale_amt,prdt.gender 
FROM ax.vw_f_scsshopweek vfs
INNER JOIN ax.vw_d_prdt prdt ON vfs.scskey  = prdt.scskey
WHERE vfs.brand = 'X' AND vfs.weekkey BETWEEN '2019-01-01' AND '2020-12-31' AND prdt.item ='SH'
'''

cur.execute(total_shop_stock_ori)
total_shop_stock_ori = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 199.19167375564575


In [12]:
total_shop_stock_ori = pd.DataFrame(total_shop_stock_ori)
total_shop_stock_ori.columns = ['weekkey','shopkey','partkey','scskey','brand','season','partcode','color','siz','week_takeout_qty','takeout_qty' ,'week_sale_qty' ,'sale_qty','week_sale_amt','stock_qty' ,'sale_amt','gender']

In [13]:
total_shop_stock = total_shop_stock_ori

In [ ]:
# total_shop_stock.to_csv("total_shop_stock.csv")

In [14]:
# 모든 매장 정보

start = time.time()

shop_info = '''
SELECT shopkey ,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
FROM ax.vw_d_shop
WHERE type_ax_zone_nm ='국내' AND chnl_type_sfdc='오프라인'
;
'''

cur.execute(shop_info)
shop_info = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 0.724820613861084


In [15]:
shop_info_pd = pd.DataFrame(shop_info)
shop_info_pd.columns = ['shopkey' ,'shop_nm','shop_nm_sfdc','repr_shopcode','type_shop_nm','type_contract_nm','type_zone_nm','region_nm']

In [16]:
# shop_info_pd.to_csv('shop_info_pd.csv')

In [36]:
# 국내 / 오프라인 / 아울렛,상설 x / 디스커버리 / 2019년 - 2020년 전 기간 운영한 매장 

start = time.time()

shop_list = '''
SELECT shop_2019.shopkey 
FROM(SELECT *
FROM ax.vw_d_shop vds
WHERE type_ax_zone_nm ='국내' AND  vds.chnl_type_sfdc ='오프라인' AND shop_nm NOT LIKE '%아울렛%' AND shop_type_sfdc NOT LIKE '%상설%' AND type_zone_nm='백화점'  AND vds.brand = 'X' AND (closedate > '2019-12-31' OR closedate is NULL ) AND opendate < '2019-01-01'
LIMIT 1000) AS shop_2019
INNER JOIN (SELECT *
FROM ax.vw_d_shop vds
WHERE type_ax_zone_nm ='국내' AND  vds.chnl_type_sfdc ='오프라인' AND shop_nm NOT LIKE '%아울렛%' AND shop_type_sfdc NOT LIKE '%상설%' AND type_zone_nm='백화점' AND vds.brand = 'X' AND (closedate > '2020-12-31' OR closedate is NULL ) AND opendate < '2020-01-01'
) AS shop_2020
ON shop_2019.shopkey =shop_2020.shopkey
'''

cur.execute(shop_list)
shop_list = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 11.199221134185791


In [37]:
shop_list_pd = pd.DataFrame(shop_list)
shop_list_pd.columns = ['shopkey']

In [38]:
shop_list_pd

,shopkey
0,X10056
1,X10063
2,X10050
3,X10019
4,X10036
5,X10078
6,X10077
7,X10079
8,X10059
9,X10065


In [19]:
# shop_list_pd.to_csv("shop_list.csv")

In [20]:
# 2019년 버킷디워커 물류 수량
start = time.time()

bk_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270') AND weekkey BETWEEN To_date('2018-12-31', 'YYYY-MM-DD') AND To_date('2019-12-31', 'YYYY-MM-DD')


'''


cur.execute(bk_stock_scsweek)
bk_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

bk_stock_scsweek_pd = pd.DataFrame(bk_stock_scsweek)
bk_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.14969110488891602


In [21]:
# bk_stock_scsweek_pd.to_csv('bk_stock_scsweek_pd.csv')

In [22]:
# 2020년 v2 버킷디워커 물류 수량
start = time.time()

v2_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270') AND weekkey BETWEEN To_date('2019-11-18', 'YYYY-MM-DD') AND To_date('2020-12-31', 'YYYY-MM-DD')


'''


cur.execute(v2_stock_scsweek)
v2_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_stock_scsweek_pd = pd.DataFrame(v2_stock_scsweek)
v2_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.1501302719116211


In [23]:
# v2_stock_scsweek_pd.to_csv('v2_stock_scsweek_pd.csv')

# #전처리 작업

# 상품관련 주차 채워주는 로직

In [7]:
df_INNER_JOIN = pd.merge(shop_list_pd, total_shop_stock, left_on='shopkey', right_on='shopkey', how='inner')

In [8]:
df_INNER_JOIN['weekkey'] = pd.to_datetime(df_INNER_JOIN['weekkey'])

In [31]:
shop_stock_2019 = df_INNER_JOIN[(df_INNER_JOIN['weekkey']< '20200101') & (df_INNER_JOIN['weekkey']> '20181231') ]
shop_stock_2020 = df_INNER_JOIN[df_INNER_JOIN['weekkey']> '20191231' ]

In [32]:
# 실제 매장 입고량과 판매량 구하는 로직(2019)
shop_stock_key_2019 = shop_stock_2019['shopkey'].unique()

total_sum_salty_2019 = []
total_sum_takeout_2019 = []
total_return_salty_2019=[]
total_sum_take_ware_2019=[]

for i in trange(0,len(shop_stock_key_2019)):
    takeout_qty = 0
    sale_qty = 0
    take_ware = 0
    return_salty=0
    
    shop_stock= shop_stock_2019[shop_stock_2019['shopkey']==shop_stock_key_2019[i]]
    shop_scs_key_2019 = shop_stock['scskey'].unique()
    for j in shop_scs_key_2019:
        takeout_qty+= shop_stock[shop_stock['scskey']==j]['takeout_qty'].max()
        sale_qty+= shop_stock[shop_stock['scskey']==j]['sale_qty'].max()
        take_ware += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_takeout_qty']<0)]['week_takeout_qty'].sum())
        return_salty += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_sale_qty']<0)]['week_sale_qty'].sum())
    total_sum_takeout_2019.append(takeout_qty)
    total_sum_salty_2019.append(sale_qty)
    total_sum_take_ware_2019.append(take_ware)
    total_return_salty_2019.append(return_salty)
df3 = pd.DataFrame([ x for x in zip(shop_stock_key_2019,total_sum_salty_2019,total_sum_takeout_2019,total_sum_take_ware_2019,total_return_salty_2019)])
df3.columns = ['shopkey','salty2019','takeout2019','take_ware2019','return_salty2019']
print(df3)

100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [09:08<00:00,  3.84s/it]

    shopkey  salty2019  takeout2019  take_ware2019  return_salty2019
0    X10082       2271         3974          -2678               -76
1    X50095       1230         2505           -996               -67
2    X50106       1735         3186          -1027              -117
3    X50102       3269         4813          -1258              -508
4    X50127       2846         4506          -1027              -145
5    X50020       1914         3444          -1126               -78
6    X50118       2002         3716          -1008               -92
7    X10085       3620         4808           -791              -105
8    X50059       1609         3006           -892               -30
9    X10018       5823         7260          -1072              -211
10   X30030       3549         5122           -976               -76
11   X10019       2462         3772          -1087               -80
12   X10047       1479         3481          -1678               -33
13   X10055       6207         823

In [34]:
# 실제 매장 입고량과 판매량 구하는 로직(2020)

shop_stock_key_2020 = shop_stock_2020['shopkey'].unique()

total_sum_salty2020 = []
total_sum_takeout2020 = []
total_return_salty2020=[]
total_sum_take_ware2020=[]

for i in trange(0,len(shop_stock_key_2020)):
    takeout_qty = 0
    sale_qty = 0
    take_ware = 0
    return_salty=0
    
    shop_stock= shop_stock_2020[shop_stock_2020['shopkey']==shop_stock_key_2020[i]]
    shop_scs_key_2020 = shop_stock['scskey'].unique()
    for j in shop_scs_key_2020:
        takeout_qty+= shop_stock[shop_stock['scskey']==j]['takeout_qty'].max()
        sale_qty+= shop_stock[shop_stock['scskey']==j]['sale_qty'].max()
        take_ware += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_takeout_qty']<0)]['week_takeout_qty'].sum())
        return_salty += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_sale_qty']<0)]['week_sale_qty'].sum())
    total_sum_takeout2020.append(takeout_qty)
    total_sum_salty2020.append(sale_qty)
    total_sum_take_ware2020.append(take_ware)
    total_return_salty2020.append(return_salty)
df4 = pd.DataFrame([ x for x in zip(shop_stock_key_2020,total_sum_salty2020,total_sum_takeout2020,total_sum_take_ware2020,total_return_salty2020)])
df4.columns = ['shopkey','salty2020','takeout2020','take_ware2020','return_salty2020']
df4

100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [10:15<00:00,  4.31s/it]


,shopkey,salty2020,takeout2020,take_ware2020,return_salty2020
0,X10082,3930,5814,-2171,-71
1,X50095,1804,2950,-843,-64
2,X50106,3069,4530,-978,-102
3,X50102,3635,5251,-1222,-119
4,X50127,4000,5654,-1214,-96
5,X50020,2822,4187,-879,-104
6,X50118,3348,5076,-1150,-130
7,X10085,3233,4604,-932,-153
8,X50059,1910,3160,-927,-30
9,X10018,8503,10473,-1301,-288


In [33]:
# buket
department_weeks_mean = shop_weeks_df[shop_weeks_df['type_zone_nm']=='백화점'].groupby('weekkey').mean()
agency_weeks_mean = shop_weeks_df[shop_weeks_df['type_zone_nm']=='대리점'].groupby('weekkey').mean()

NameError: name 'shop_weeks_df' is not defined

In [ ]:
department_weeks_mean = department_weeks_mean.reset_index()
agency_weeks_mean = agency_weeks_mean.reset_index()
department_weeks_mean['weekkey'] = pd.to_datetime(department_weeks_mean['weekkey'])
agency_weeks_mean['weekkey'] = pd.to_datetime(agency_weeks_mean['weekkey'])

In [ ]:
department_weeks_mean = department_weeks_mean.set_index('weekkey')
agency_weeks_mean = agency_weeks_mean.set_index('weekkey')

In [ ]:
# buket_v2
v2_department_weeks_mean = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점'].groupby('weekkey').mean()
v2_agency_weeks_mean = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='대리점'].groupby('weekkey').mean()

In [ ]:
v2_department_weeks_mean = v2_department_weeks_mean.reset_index()
v2_agency_weeks_mean = v2_agency_weeks_mean.reset_index()
v2_department_weeks_mean['weekkey'] = pd.to_datetime(v2_department_weeks_mean['weekkey'])
v2_agency_weeks_mean['weekkey'] = pd.to_datetime(v2_agency_weeks_mean['weekkey'])

In [ ]:
v2_department_weeks_mean = v2_department_weeks_mean.set_index('weekkey')
v2_agency_weeks_mean = v2_agency_weeks_mean.set_index('weekkey')

In [ ]:
shop_total_sales_info = pd.merge(shop_total_sales,shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
shop_total_sales_info

In [61]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]

# 버킷디워커 물류 재고량
bk_stock_scsweek_pd = pd.read_csv("bk_stock_scsweek_pd.csv")
bk_stock_scsweek_pd = bk_stock_scsweek_pd.iloc[ :,1:]

# 버킷디워커 물류 재고량
v2_stock_scsweek_pd = pd.read_csv("v2_stock_scsweek_pd.csv")
v2_stock_scsweek_pd = v2_stock_scsweek_pd.iloc[ :,1:]

# 매장 입출판재

In [10]:
bucket_dwalker = df_INNER_JOIN[(df_INNER_JOIN['partkey'].str.contains("X19SDXSH09911")) & (df_INNER_JOIN['weekkey'] < '2019-12-31' ) ]

In [12]:
bucket_dwalker_v2 = df_INNER_JOIN[df_INNER_JOIN['partkey'].str.contains("X20SDXSHA5011|X20SDXSHA3011|X20SDXSHA1011|X20SDXSHA1011") & (df_INNER_JOIN['weekkey'] < '2020-12-31' )]

In [59]:
# 빠져있는 주차 채워주는 로직

def zero_fill(shopkey, base, scskey_list,item):
    total=[]
    for col in base.columns:
        base[col].values[:] = 0
    for scskey in scskey_list:
        for shop in shopkey:
            frame = base.reset_index()
            target = item[(item['scskey']==scskey) & (item['shopkey']==shop )].groupby('weekkey').sum().reset_index()
            target = target.reset_index()

            for i in range(0,len(target)):
                for j in range(0,len(frame)):
                        if target.iloc[i].weekkey == frame.iloc[j].weekkey:
                            frame.iloc[j] = target.iloc[i]
                            break
            for i in range(1,len(frame)):
                if frame.iloc[i].takeout_qty ==0 and frame.iloc[i].sale_qty==0:
                    frame.iloc[i] = [ frame.iloc[i]['weekkey'], 0, frame.iloc[i-1]['takeout_qty'], 0, frame.iloc[i-1]['sale_qty'], frame.iloc[i-1]['stock_qty']]
            frame['shopkey'] =shop
            frame['ratio'] = 0
            for i in range(0,len(frame)):
                total_sale_qty= frame['week_sale_qty'].sum()
                a = frame.iloc[i]
                if a['sale_qty']==0 :
                    frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0]
                else:
                    if a['sale_qty'] / total_sale_qty > 1:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1]
                    else:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]
            else:
                frame['scskey']=scskey
            frame = pd.merge(frame, shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
            total.append(frame)
    return total


In [17]:
scskey_list = ['X19SDXSH09911BK230','X19SDXSH09911BK235','X19SDXSH09911BK240','X19SDXSH09911BK260','X19SDXSH09911BK265','X19SDXSH09911BK270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911BK240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)

<ipython-input-13-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [20]:
shop_weeks_buket_df = pd.concat(shop_weeks_buket)
# shop_weeks_buket_df.to_csv('shop_weeks_buket_df.csv')

In [ ]:
scskey_list = ['X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA5011OW240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)

In [ ]:
shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)
# shop_weeks_buket_v2_df.to_csv('shop_weeks_buket_v2.csv')

In [28]:
bucket_dwalker.groupby('color').sum()

,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt
color,,,,,,,,
BE,8302620,32827,553930,30325,436401,3738871787.00000,117529,54022326614.00000
BK,4575365,11026,113255,10023,84142,1233573459.00000,29113,10392705940.00000
GR,4386530,13603,110822,8746,66159,1062545040.00000,44663,8099859117.00000
WH,2382520,5602,40954,5454,31052,675870403.00000,9902,3855050224.00000


In [25]:
bucket_dwalker_v2.groupby('color').sum()

,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt
color,,,,,,,,
BE,9416575,35644,597617,28789,490670,3836778535.00000,106947,65569353399.00000
OW,12628725,48516,977846,42821,808983,5711217015.00000,168863,108264290796.00000
WH,7990910,27064,398665,22154,315019,2940333458.00000,83646,41940362006.00000


In [27]:
bucket_dwalker_v2.head(100)

,shopkey,weekkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
2560,X10052,2019-11-18,X20SDXSHA1011,X20SDXSHA1011WH225,X,20S,DXSHA1011,WH,225,1,1,0,0,0.00000,1,0.00000,X
2561,X10052,2019-12-09,X20SDXSHA1011,X20SDXSHA1011WH225,X,20S,DXSHA1011,WH,225,1,2,0,1,0.00000,1,139000.00000,X
2562,X10052,2020-05-25,X20SDXSHA1011,X20SDXSHA1011WH225,X,20S,DXSHA1011,WH,225,-1,1,0,1,0.00000,0,139000.00000,X
2563,X10052,2019-11-18,X20SDXSHA1011,X20SDXSHA1011WH230,X,20S,DXSHA1011,WH,230,1,1,0,0,0.00000,1,0.00000,X
2564,X10052,2020-02-03,X20SDXSHA1011,X20SDXSHA1011WH230,X,20S,DXSHA1011,WH,230,1,4,1,2,139000.00000,2,278000.00000,X
2565,X10052,2020-02-17,X20SDXSHA1011,X20SDXSHA1011WH230,X,20S,DXSHA1011,WH,230,3,7,1,3,118150.00000,4,396150.00000,X
2566,X10052,2020-05-25,X20SDXSHA1011,X20SDXSHA1011WH230,X,20S,DXSHA1011,WH,230,-3,5,0,4,0.00000,1,535150.00000,X
2567,X10052,2019-11-18,X20SDXSHA1011,X20SDXSHA1011WH250,X,20S,DXSHA1011,WH,250,1,1,1,1,139000.00000,0,139000.00000,X
2568,X10052,2020-05-25,X20SDXSHA1011,X20SDXSHA1011WH250,X,20S,DXSHA1011,WH,250,-1,1,0,1,0.00000,0,139000.00000,X
2574,X10052,2019-11-18,X20SDXSHA3011,X20SDXSHA3011BE265,X,20S,DXSHA3011,BE,265,1,1,0,0,0.00000,1,0.00000,X


In [11]:
# 2020년 v2 버킷디워커 물류 수량
start = time.time()

test3 = '''
SELECT sales.shopkey ,sales.partkey,sales.scskey,sales.saledate,sales.cust_type_nm,sales.season,sales.partcode,sales.color,sales.siz,sales.cid,sales.qty,sales.amt_act,sales.amt_tag,prdt.sellprice,prdt.partcode_nm
FROM ax.mvw_f_sales sales
INNER JOIN ax.vw_d_prdt prdt ON sales.scskey  = prdt.scskey
WHERE sales.brand = 'X' AND sales.saledate BETWEEN To_date('2021-01-01', 'YYYY-MM-DD') AND
                           To_date('2021-04-22', 'YYYY-MM-DD') AND item = 'SH'
;


'''


cur.execute(test3)
test3 = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

test3 = pd.DataFrame(test3)


time : 14.200255393981934


In [12]:
test3.columns = ['shopkey' ,'partkey', 'scskey' ,'saledate', 'cust_type_nm' ,'season','partcode','color','siz','cid','qty','amt_act','amt_tag','sellprice','partcode_nm']

In [13]:
test3.groupby('partkey').count().sort_values('shopkey',ascending='False')

,shopkey,scskey,saledate,cust_type_nm,season,partcode,color,siz,cid,qty,amt_act,amt_tag,sellprice,partcode_nm
partkey,,,,,,,,,,,,,,
X18SDXSH70831,1,1,1,1,1,1,1,1,0,1,1,1,1,1
X19SDXSH73931,1,1,1,1,1,1,1,1,0,1,1,1,1,1
X18FDXSH13841,1,1,1,1,1,1,1,1,0,1,1,1,1,1
X19FDXSH73961,1,1,1,1,1,1,1,1,0,1,1,1,1,1
X18FDXSH41841,1,1,1,1,1,1,1,1,1,1,1,1,1,1
X18FDXSH42841,1,1,1,1,1,1,1,1,1,1,1,1,1,1
X18SDXSH30811,1,1,1,1,1,1,1,1,0,1,1,1,1,1
X20SDXSHW2031,2,2,2,2,2,2,2,2,0,2,2,2,2,2
X18FDXSH08841,2,2,2,2,2,2,2,2,0,2,2,2,2,2


In [17]:
test4 = test3[test3['partkey']=='X21SDXSHA1111']

In [41]:
test4

,shopkey,partkey,scskey,saledate,cust_type_nm,season,partcode,color,siz,cid,qty,amt_act,amt_tag,sellprice,partcode_nm
3,X50001,X21SDXSHA1111,X21SDXSHA1111OW280,2021-01-22,소매,21S,DXSHA1111,OW,280,None,1,139000.00,139000,139000,버킷 디워커 V2
14,X40002,X21SDXSHA1111,X21SDXSHA1111BK230,2021-02-17,소매,21S,DXSHA1111,BK,230,None,1,139000.00,139000,139000,버킷 디워커 V2
21,X10007,X21SDXSHA1111,X21SDXSHA1111IV270,2021-02-10,소매,21S,DXSHA1111,IV,270,X8751292,1,132050.00,139000,139000,버킷 디워커 V2
28,X30004,X21SDXSHA1111,X21SDXSHA1111CR245,2021-02-21,소매,21S,DXSHA1111,CR,245,X8563766,1,129000.00,139000,139000,버킷 디워커 V2
32,X50151,X21SDXSHA1111,X21SDXSHA1111WW240,2021-02-27,소매,21S,DXSHA1111,WW,240,X2821218,1,139000.00,139000,139000,버킷 디워커 V2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333945,X10043,X21SDXSHA1111,X21SDXSHA1111WH265,2021-03-06,소매,21S,DXSHA1111,WH,265,None,1,139000.00,139000,139000,버킷 디워커 V2
333962,X50103,X21SDXSHA1111,X21SDXSHA1111WW265,2021-03-26,소매,21S,DXSHA1111,WW,265,None,1,139000.00,139000,139000,버킷 디워커 V2
333977,X80005,X21SDXSHA1111,X21SDXSHA1111IV245,2021-03-26,소매,21S,DXSHA1111,IV,245,None,1,139000.00,139000,139000,버킷 디워커 V2
333994,X80008,X21SDXSHA1111,X21SDXSHA1111WH235,2021-01-05,소매,21S,DXSHA1111,WH,235,None,1,139000.00,139000,139000,버킷 디워커 V2


In [19]:
test4.groupby('color').count()

,shopkey,partkey,scskey,saledate,cust_type_nm,season,partcode,siz,cid,qty,amt_act,amt_tag,sellprice,partcode_nm
color,,,,,,,,,,,,,,
BE,2627,2627,2627,2627,2627,2627,2627,2627,910,2627,2627,2627,2627,2627
BK,3319,3319,3319,3319,3319,3319,3319,3319,938,3319,3319,3319,3319,3319
CR,4534,4534,4534,4534,4534,4534,4534,4534,2163,4534,4534,4534,4534,4534
IV,6380,6380,6380,6380,6380,6380,6380,6380,2740,6380,6380,6380,6380,6380
LG,1962,1962,1962,1962,1962,1962,1962,1962,865,1962,1962,1962,1962,1962
OW,8429,8429,8429,8429,8429,8429,8429,8429,3552,8429,8429,8429,8429,8429
WH,6028,6028,6028,6028,6028,6028,6028,6028,2180,6028,6028,6028,6028,6028
WW,7058,7058,7058,7058,7058,7058,7058,7058,3348,7058,7058,7058,7058,7058


In [20]:
# 매장 입출판재 

start = time.time()

total_shop_stock_ori2 = '''
SELECT vfs.weekkey,vfs.shopkey ,vfs.partkey,vfs.scskey,vfs.brand,vfs.season,vfs.partcode,vfs.color,vfs.siz,vfs.week_takeout_qty,vfs.takeout_qty ,vfs.week_sale_qty ,vfs.sale_qty,vfs.week_sale_amt,vfs.stock_qty ,vfs.sale_amt,prdt.gender 
FROM ax.vw_f_scsshopweek vfs
INNER JOIN ax.vw_d_prdt prdt ON vfs.scskey  = prdt.scskey
WHERE vfs.brand = 'X' AND vfs.weekkey BETWEEN '2020-12-01' AND '2021-12-31' AND prdt.item ='SH'
'''

cur.execute(total_shop_stock_ori2)
total_shop_stock_ori2 = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 21.53980565071106


In [21]:
total_shop_stock_ori2 = pd.DataFrame(total_shop_stock_ori2)
total_shop_stock_ori2.columns = ['weekkey','shopkey','partkey','scskey','brand','season','partcode','color','siz','week_takeout_qty','takeout_qty' ,'week_sale_qty' ,'sale_qty','week_sale_amt','stock_qty' ,'sale_amt','gender']

In [110]:
total_shop_stock_ori2.to_csv('total_shop_stock_ori2.csv')

In [100]:
shop_weeks_buket_v2_2021_df = pd.merge(shop_list_pd, total_shop_stock_ori2, left_on='shopkey', right_on='shopkey', how='inner')

In [101]:
shop_weeks_buket_v2_2021_df.to_csv('shop_weeks_buket_v2_2021_df.csv')

In [29]:
total_shop_stock_ori2[(total_shop_stock_ori2['scskey']=='X21SDXSHA1111BK260')].sort_values('weekkey')

,weekkey,shopkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
53532,2020-12-14,X90019,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,40,40,0,0,0.00,40,0.00,X
55420,2020-12-14,X30004,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,40,40,0,0,0.00,40,0.00,X
297499,2020-12-21,X10063,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,1,1,0,0,0.00,1,0.00,X
618187,2020-12-21,X10064,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,1,1,0,0,0.00,1,0.00,X
53533,2020-12-21,X90019,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,-1,39,0,0,0.00,39,0.00,X
135583,2020-12-21,X50156,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,2,2,0,0,0.00,2,0.00,X
126354,2020-12-21,X50140,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,1,1,0,0,0.00,1,0.00,X
446462,2020-12-21,X50053,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,2,2,0,0,0.00,2,0.00,X
375279,2020-12-21,X50163,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,3,3,1,1,139000.00,2,139000.00,X
616077,2020-12-21,X10055,X21SDXSHA1111,X21SDXSHA1111BK260,X,21S,DXSHA1111,BK,260,2,2,0,0,0.00,2,0.00,X


In [88]:
# BE
scskey_list = ['X21SDXSHA1111BE230','X21SDXSHA1111BE235','X21SDXSHA1111BE240','X21SDXSHA1111BE260','X21SDXSHA1111BE260','X21SDXSHA1111BE270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
be_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

In [89]:
# BK
scskey_list = ['X21SDXSHA1111BK230','X21SDXSHA1111BK235','X21SDXSHA1111BK240','X21SDXSHA1111BK260','X21SDXSHA1111BK260','X21SDXSHA1111BK270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
bk_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [90]:
# CR
scskey_list = ['X21SDXSHA1111CR230','X21SDXSHA1111CR235','X21SDXSHA1111CR240','X21SDXSHA1111CR260','X21SDXSHA1111CR260','X21SDXSHA1111CR270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
cr_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [91]:
# IV
scskey_list = ['X21SDXSHA1111IV230','X21SDXSHA1111IV235','X21SDXSHA1111IV240','X21SDXSHA1111IV260','X21SDXSHA1111IV260','X21SDXSHA1111IV270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
iv_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [92]:
# LG
scskey_list = ['X21SDXSHA1111LG230','X21SDXSHA1111LG235','X21SDXSHA1111LG240','X21SDXSHA1111LG260','X21SDXSHA1111LG260','X21SDXSHA1111LG270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
lg_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [93]:
# ow
scskey_list = ['X21SDXSHA1111OW230','X21SDXSHA1111OW235','X21SDXSHA1111OW240','X21SDXSHA1111OW260','X21SDXSHA1111OW260','X21SDXSHA1111OW270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
ow_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-59-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [94]:
# WH
scskey_list = ['X21SDXSHA1111WH230','X21SDXSHA1111WH235','X21SDXSHA1111WH240','X21SDXSHA1111WH260','X21SDXSHA1111WH260','X21SDXSHA1111WH270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
wh_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [95]:
# WW
scskey_list = ['X21SDXSHA1111WW230','X21SDXSHA1111WW235','X21SDXSHA1111WW240','X21SDXSHA1111WW260','X21SDXSHA1111WW260','X21SDXSHA1111WW270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
ww_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-59-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-59-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [102]:
be_shop_weeks_buket_2021_df.to_csv('be_shop_weeks_buket_2021_df.csv')

In [103]:
bk_shop_weeks_buket_2021_df.to_csv('bk_shop_weeks_buket_2021_df.csv')

In [104]:
cr_shop_weeks_buket_2021_df.to_csv('cr_shop_weeks_buket_2021_df.csv')

In [105]:
iv_shop_weeks_buket_2021_df.to_csv('iv_shop_weeks_buket_2021_df.csv')

In [106]:
lg_shop_weeks_buket_2021_df.to_csv('lg_shop_weeks_buket_2021_df.csv')

In [107]:
ow_shop_weeks_buket_2021_df.to_csv('ow_shop_weeks_buket_2021_df.csv')

In [108]:
wh_shop_weeks_buket_2021_df.to_csv('wh_shop_weeks_buket_2021_df.csv')

In [109]:
ww_shop_weeks_buket_2021_df.to_csv('ww_shop_weeks_buket_2021_df.csv')

In [87]:
frame = df_INNER_JOIN2[(df_INNER_JOIN2['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
frame

,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty
weekkey,,,,,
2020-12-07,66,66,7,7,59
2020-12-14,24,41,17,24,17
2020-12-21,11,40,23,31,9
2020-12-28,33,85,11,50,35
2021-01-04,66,156,16,63,93
2021-01-11,25,110,26,65,45
2021-01-18,29,117,22,76,41
2021-01-25,17,116,12,73,43
2021-02-01,30,210,31,131,79
